# LIN 371 Machine Learning for Text Analysis

# Homework 2 - due Friday Feb 16 2024 at 11:59pm

For this homework you will hand in (upload) to canvas your code named ``hw2_YourEID.ipynb``.

__Before submitting__, please reset your kernel and rerun everything from the beginning (`Kernel` >> `Restart and Run All`) and ensure your code outputs the correct answer.

A perfect solution for this homework is worth **100** points, and there is a bonus problem at the end that is worth **5 points**. For non-programming tasks, you must show work to get partial credit. For programming tasks, make sure that your code can run using Python 3.5+. If you cannot complete a problem, include as much pseudocode as possible for partial credit. However, make sure it does not have any output errors. **If there are any output errors, half of the points for that problem will be automatically deducted.**

Collaboration: you are free to discuss the homework assignments with other students and work towards solutions together.  However, all of the code you write must be your own! There is a channel on Slack where you can look for a study group.

Review extension and academic dishonesty policy here: https://jessyli.com/courses/lin371

For typing up your answers to problems 1, 2 and 3, information can be found about Markdown cells for Jupyter Notebooks here: https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html


### Please list any collaborators here:
- 


# Problem 1: Naive Bayes

We'll do a bit of manual parameter estimation here.

## **(a)** (8 points) 
Calculate the sufficient parameters for Naive Bayes using the data in the figure below, that
is, the prior class probabilities and the conditional probabilities for all of the symbols.

doc | X | Y
-- | --| --
d1 | a b b b c b | +
d2 | c a c c c b | -
d3 | c c c b | -
d4 | b a b b b | +
d5 | b c a b b | ???


In [1]:
# initialize the training data and classes
docs_train = ['a b b b c b',
              'c a c c c b',
              'c c c b',
              'b a b b b']
Y_train = [1, 0, 0, 1]

# tokenize train and test data
docs_train_tokenized = [doc.split() for doc in docs_train]


In [2]:
# create IDs for each character that will serve as columns in the matrix
char_to_col_id = {}

for doc in docs_train_tokenized:
    for char in doc:
        if char not in char_to_col_id:
            char_to_col_id[char] = len(char_to_col_id)

print(char_to_col_id)

{'a': 0, 'b': 1, 'c': 2}


In [3]:
import numpy as np 

# initialize matrix
X_train = np.zeros((len(docs_train), len(char_to_col_id)))

# column represents the word
# row represents the doc
for i, doc in enumerate(docs_train_tokenized):
    for word in doc:
        rowid = i
        colid = char_to_col_id[word]
        X_train[rowid][colid] += 1

print(X_train)

[[1. 4. 1.]
 [1. 1. 4.]
 [0. 1. 3.]
 [1. 4. 0.]]


In [4]:
from collections import defaultdict

prior = [0,0]
condprob = defaultdict(dict)

# P(c|x) = P(x|c)P(x)
    # Posterior = P(x|c) = number of times x appears in class c
    # Prior = P(x) = number of times class c occured out of all instances

for c in [0,1]:
    prior[c] = Y_train.count(c)/len(Y_train)

    # summing all items for class c
    # this will serve as the denominator for our posterior
    countallc = sum([sum(X_train[i]) for i, y in enumerate(Y_train) if y == c])

    print('class', c, 'prior =', prior[c])

    for token, tid in char_to_col_id.items():
        # sums number of time each token occurs for each class
        # wills serve as the numerator for our posterior
        count_tc = sum([X_train[i][tid] for i, y in enumerate(Y_train) if y == c])

        # populate the conditional probabilities
        # laplace smoothing, +1 to numerator, +# of tokens to denominator
        condprob[tid][c] = (count_tc + 1)/(countallc + len(char_to_col_id))
        
        print('conditional probability of', token, 'in class', c, 'is', condprob[tid][c])
    print()
    

class 0 prior = 0.5
conditional probability of a in class 0 is 0.15384615384615385
conditional probability of b in class 0 is 0.23076923076923078
conditional probability of c in class 0 is 0.6153846153846154

class 1 prior = 0.5
conditional probability of a in class 1 is 0.21428571428571427
conditional probability of b in class 1 is 0.6428571428571429
conditional probability of c in class 1 is 0.14285714285714285



## **(b)** (7 points) 
Using these, calculate the most likely class (pos or neg) for the unlabeled example (d5, labeled ???).

In [5]:
docs_test = ['b c a b b']
docs_test_tokenized = [doc.split() for doc in docs_test]

In [6]:
X_test = np.zeros((len(docs_test),len(char_to_col_id)))

for i, doc in enumerate(docs_test_tokenized):
    for word in doc:
        rowid = i
        if word in char_to_col_id:
            colid = char_to_col_id[word]
            X_test[rowid][colid] += 1

print(X_test)

[[1. 3. 1.]]


In [7]:
testprob = [0,0]

for c in [0, 1]:
    testprob[c] = prior[c]
    for i, x in enumerate(X_test[0]):
        testprob[c] = testprob[c] * (condprob[i][c] ** x)
    
    print('class', c, testprob[c])
print()

for c in [0,1]:
    print('The probability of d5 being in class', c, 'is', testprob[c]/sum(testprob))

class 0 0.0005817508005806736
class 1 0.0040663860296305115

The probability of d5 being in class 0 is 0.1251578475055869
The probability of d5 being in class 1 is 0.8748421524944131


# Problem 2: Language Modeling

In this problem, we will implement our own Shakespeare and Austen language models!

### Data

We will be working with a few different files under the `data` subdirectory. 
- `shakespeare.txt`
- `shakespeare_short.txt`
- `austen.txt`
- `austen_short.txt`
- `test_short.txt`

<mark>You should use the "short" text files to do testing and debugging</mark> as the other files will take a little while (but should be under 10 mins for a 8G RAM laptop) to run! 

### Note re NLTK

The goal of this problem is to think about how to implement language models from first principles. Thus, **the *ONLY* allowable functions from NTLK in this problem are `sent_tokenize` and `word_tokenize`.**

## (a) Clean text. (5 points) 

Create a function `sent_transform(sent_string)`, such that when given a sentence as a string, it would return a `list` of words, lower-cased. Use NLTK's `word_tokenize` function to tokenize the sentence. For now, you can use any random string to test this function.

```
>>> sent_transform("ROSALIND. Why, whither shall we go?")
['rosalind', '.', 'why', ',', 'whither', 'shall', 'we', 'go', '?']
```

In [8]:
def sent_transform(sent_string):
    ## YOUR CODE HERE
    from nltk import word_tokenize
    
    sent_tokenized = word_tokenize(sent_string.lower())
    return sent_tokenized

sent_transform("ROSALIND. Why, whither shall we go?")

['rosalind', '.', 'why', ',', 'whither', 'shall', 'we', 'go', '?']

## (b) Find n-grams. (10 points) 

Create a function `make_ngram_tuples(words, n)` that returns a sequence of all n-grams seen in the input, in order.  The function returns a sequence of tuples where each tuple is of the form `(context, word)`.  The context is a tuple of the preceding n−1 words for each word; if the number of preceding words is smaller than n−1, place a `<s>` symbol in place of each missing context.  Close each sentence with an additional token `</s>`.  You can assumen n>1, that is, we are interested in enumerating bigrams, trigrams etc, and not unigrams.

For now, you can use any random string to test this function.

```
>>> samples = [’she’, ’eats’, ’happily’ ’.’]
>>> make_ngram_tuples(samples, 2)
[((’<s>’,), ’she’), ((’she’,), ’eats’), ((’eats’,), ’happily’), ((’happily’,), ’.’), ((’.’,), ’</s>’)]
>>> make_ngram_tuples(samples, 3)

[((’<s>’, ’<s>’), ’she’), ((’<s>’, ’she’), ’eats’), ((’she’, ’eats’), ’happily’),((’eats’, ’happily’), ’.’), ((’happily’, ’.’), ’</s>’)]
```

In [9]:
def make_ngram_tuples(words, n):
    ## YOUR CODE HERE
    words.insert(0, "<s>")
    words.insert(len(words), "</s>")

    ngrams = []

    # enumerate over the words
    for idx, word in enumerate(words):
        # initialize a list to hold the prior
        prior = []
        # loop up to n + 1 to include n
        # start at 1 because 0 will always be a <s>
        for i in range(1, n + 1):
            # don't want to generate ngrams for <s>
            if idx > 0:
                # until we hit n, we want to add to the prior
                if i < n:
                    # if we reach 0 before i == n, just keep adding the 0th element
                    prior.insert(-1, words[idx - i if idx - i > 0 else 0])
                # put the ngram together and append the list
                elif i == n:
                    n_gram = (tuple(prior), word)
                    ngrams.append(n_gram)

    return ngrams

make_ngram_tuples(['she', 'eats', 'happily', '.'], 2)

[(('<s>',), 'she'),
 (('she',), 'eats'),
 (('eats',), 'happily'),
 (('happily',), '.'),
 (('.',), '</s>')]

## (c)  Building an n-gram language model

We are now ready to estimate a bigram language model from the training data.  We will use **Laplace smoothing (add-1)**.

### (c1) Process the training file (8 points)

We will first need to transform a file of plain text into a list of tokenized "sentences", where each "sentence" is a list of *lower-cased* words. Implement a function `process_text(textfile)` to do so. Assume that the path of `textfile` is of a form like `data/austen_short.txt`, i.e., relative paths. Make sure that you call `sent_transform` in this function.

```
>>> processed_sents = process_text("data/austen_short.txt")
>>> print(processed_sents[10])
['he', 'considered', 'the', 'blessing', 'of', 'beauty', 'as', 'inferior', 'only', 'to', 'the', 'blessing', 'of', 'a', 'baronetcy', ';', 'and', 'the', 'sir', 'walter', 'elliot', ',', 'who', 'united', 'these', 'gifts', ',', 'was', 'the', 'constant', 'object', 'of', 'his', 'warmest', 'respect', 'and', 'devotion', '.']
```

In [10]:
def process_text(textfile):

    from nltk import sent_tokenize
    # read the file
    f = open(textfile).read()
    # tokenize the sentences
    processed_sents = sent_tokenize(f)
    # process each tokenized sentence using sent_transform
    processed_sents = [sent_transform(sent) for sent in processed_sents]

    return processed_sents

processed_sents = process_text("data/austen_short.txt")
print(processed_sents[10])

['he', 'considered', 'the', 'blessing', 'of', 'beauty', 'as', 'inferior', 'only', 'to', 'the', 'blessing', 'of', 'a', 'baronetcy', ';', 'and', 'the', 'sir', 'walter', 'elliot', ',', 'who', 'united', 'these', 'gifts', ',', 'was', 'the', 'constant', 'object', 'of', 'his', 'warmest', 'respect', 'and', 'devotion', '.']


### (c2) Vocabulary (8 points).

The language model ought to be able to handle words not seen in the training data.  Such words will most certainly appear if the LM is used in some application to estimate the likelihood of new data.   There  are  many  ways  to  incorporate  unknown  vocabulary  in  a  language  model.   In  this problem, we will take all words that appear only once and replace them with the symbol `<UNK>`. Thus, at test time, if we encounter an out-of-vocabulary word, we can resort to replacing the word with `<UNK>` which has well-formed probabilities.

First, implement a function `get_vocab(tokenized_sents)` where the parameter `tokenized_sents` is a list of tokenized "sentences" (where each "sentence" is a list of lower-cased words) returned by the function `process_text`. The function `get_vocab` will return a `set` of all unique words in `tokenized_sents` that appeared more than once.
Note: do not forget to add three extra tokens to the vocabulary: `<s>`, `</s>`, and `<UNK>`.



```
>>> vocab = get_vocab(processed_sents)
>>> print(len(vocab))
302
```

In [11]:
def get_vocab(tokenized_sents):

    # initialize a dictionary with two items
    word_dict = {'<s>': 0, '</s>': 0}

    # loop through the sentences
    for sent in tokenized_sents:
        # each sentence adds one to our start and end paddings
        word_dict['<s>'] += 1
        word_dict['</s>'] += 1
        # count the number of occurences of each word
        for word in sent:
            if word in word_dict:
                word_dict[word] += 1
            else:
                word_dict[word] = 1
    
    # add UNK to the dict with a value of the number of words that only appear once
    word_dict['<UNK>'] = len([key for key in word_dict.keys() if word_dict[key] == 1])

    # only return the set of keys that have a value greater than one
    return set((key for key in word_dict.keys() if word_dict[key] > 1))

vocab = get_vocab(processed_sents)
print(len(vocab))

302


### (c3) Process unknown words (8 points).

Write a function `process_unk(tokenized_sents, vocab)` that takes in (1) tokenized sentences returned by `process_text`, and (2) a vocabulary (set of words) returned by `get_vocab`. The function returns a list of tokenized sentences that is the same as `tokenized_sents` except that all words appearing only once will be replaced by the symbol `<UNK>`.

```
>>> print(processed_sents[3])
['of', 'south', 'park', ',', 'in', 'the', 'county', 'of', 'gloucester', ',', 'by', 'which', 'lady', '(', 'who', 'died', '1800', ')', 'he', 'has', 'issue', 'elizabeth', ',', 'born', 'june', '1', ',', '1785', ';', 'anne', ',', 'born', 'august', '9', ',', '1787', ';', 'a', 'still-born', 'son', ',', 'november', '5', ',', '1789', ';', 'mary', ',', 'born', 'november', '20', ',', '1791', '.', "''"]
>>> processed_unk = process_unk(processed_sents, vocab)
>>> print(processed_unk[3])
['of', '<UNK>', '<UNK>', ',', 'in', 'the', 'county', 'of', '<UNK>', ',', 'by', 'which', 'lady', '(', 'who', 'died', '<UNK>', ')', 'he', 'has', '<UNK>', 'elizabeth', ',', 'born', '<UNK>', '1', ',', '<UNK>', ';', 'anne', ',', 'born', '<UNK>', '<UNK>', ',', '<UNK>', ';', 'a', '<UNK>', 'son', ',', 'november', '<UNK>', ',', '<UNK>', ';', 'mary', ',', 'born', 'november', '<UNK>', ',', '<UNK>', '.', "''"]
```

In [25]:
def process_unk(tokenized_sents, vocab):
    ## YOUR CODE HERE
    processed_unk = []

    for sent in tokenized_sents:
        processed_unk_sent = []
        for word in sent:
            if word in vocab:
                processed_unk_sent.append(word)
            else:
                processed_unk_sent.append('<UNK>')
        processed_unk.append(processed_unk_sent)

    return processed_unk

print(processed_sents[3])
processed_unk = process_unk(processed_sents, vocab)
print(processed_unk[3])

['of', 'south', 'park', ',', 'in', 'the', 'county', 'of', 'gloucester', ',', 'by', 'which', 'lady', '(', 'who', 'died', '1800', ')', 'he', 'has', 'issue', 'elizabeth', ',', 'born', 'june', '1', ',', '1785', ';', 'anne', ',', 'born', 'august', '9', ',', '1787', ';', 'a', 'still-born', 'son', ',', 'november', '5', ',', '1789', ';', 'mary', ',', 'born', 'november', '20', ',', '1791', '.', "''"]
['of', '<UNK>', '<UNK>', ',', 'in', 'the', 'county', 'of', '<UNK>', ',', 'by', 'which', 'lady', '(', 'who', 'died', '<UNK>', ')', 'he', 'has', '<UNK>', 'elizabeth', ',', 'born', '<UNK>', '1', ',', '<UNK>', ';', 'anne', ',', 'born', '<UNK>', '<UNK>', ',', '<UNK>', ';', 'a', '<UNK>', 'son', ',', 'november', '<UNK>', ',', '<UNK>', ';', 'mary', ',', 'born', 'november', '<UNK>', ',', '<UNK>', '.', "''"]


### (c4) N-gram frequencies (10 points).

We now need to get the frequency counts -- which will allow us to calculate the conditional frequency distribution for our language model. Write a function `get_freq_dist(tokenized_sents, n)` that takes in (1) a list of tokenized sentences (such as one returned by `process_unk`), and (2) the number `n` that denotes the order of the n-gram (`n=2` means bigram, `n=3` means trigram, etc). The function will return a dictionary `freqdict` (`freqdict` can be a `Counter`) such that `freqdict[context][word]` records the number of times `word` follows `context`. You can think of `context` as the first element of a tuple in a list returned by `make_ngram_tuples`, and `word` as the second element of that tuple.

```
>>> freqdict = get_freq_dict(processed_unk, 2)
>>> print(freqdict[('of',)])
Counter({'<UNK>': 25, 'the': 17, 'his': 6, 'her': 6, 'kellynch': 4, 'a': 4, 'being': 4, 'their': 3, 'it': 3, 'charles': 2, 'somerset': 2, 'any': 2, 'very': 2, 'mind': 2, 'life': 2, 'all': 2, 'mr': 2, 'them': 2, 'himself': 1, 'mary': 1, 'high': 1, 'baronet': 1, 'sir': 1, 'character': 1, 'real': 1, 'ever': 1, 'respectability': 1, '.': 1, 'youth': 1, 'and': 1, 'elliot': 1, 'inferior': 1, 'again': 1, 'economy': 1, 'which': 1, ';': 1})
```

In [13]:
def get_freq_dict(tokenized_sents, n):
    ## YOUR CODE HERE
    return {}

freqdict = get_freq_dict(processed_unk, 2)
print(freqdict[('of',)])

KeyError: ('of',)

### (c5) The langauge model

We are now ready to put everything together and make our langauge model! Below we have written the function `build_ngram_model(textfile, n)` that takes in a text file, the value `n` that signals the order of our n-gram, and returns a language model in the form of a `namedtuple`. All we need to do is to call the various functions you've implemented so far in (c)! (**No implementation required here**).

A `namedtuple` is a versatile data structure that allows one to associate multiple data fields with one variable. Below, we created a `namedtuple` called `LanguageModel`; this `LanguageModel` consists of the following information: the n-gram order `n`, the frequency distribution dictionary `fd`, and the vocabulary (as a `set` of words) `vocab`. Now, after we make a `LanguageModel`, we will be able to access these fields using the `dot` operator, for example, `toy_lm.vocab`.

In [ ]:
## DO NOT MODIFY THIS CELL, but you need to run it.

from collections import namedtuple

def build_ngram_model(textfile, n):
    LanguageModel = namedtuple('LanguageModel', ['n', 'fd', 'vocab'])
    psents = process_text(textfile)
    vocab = get_vocab(psents)
    psentsunk = process_unk(psents, vocab)
    fd = get_freq_dict(psentsunk, n)
    return LanguageModel(n, fd, vocab)

toy_lm = build_ngram_model("data/austen_short.txt", 2)

NameError: name 'process_unk' is not defined

### (c6) Log conditional probabilities. (10 points)

The heart of the language model above is just the frequency dictionary. To make our model functional, we will need to use the frequency dictionary to calculate (log) conditional probabilities. Write a function `log_prob(lm, context, word)` that takes in a language model `lm`, `context` in the form of a tuple, and a `word`. The function returns the *add-1 (Laplacian) smoothed* conditional probability values `log P(word|context)`.

We would like to calculate log probabilities, instead of raw probability values, because ultipmately we will use the language model to calculate the likelihood of a text. Multiplying many very small raw probability values may result in underflow issues (and inaccuracies) in any programming language.

You will need to get the size of the vocabulary when writing this function. Keep in mind that the `<UNK>`, `<s>`, and `</s>` symbols should all be a part of your vocabulary.

*Please use log with base 2 for this problem!*


```
>>> log_prob(toy_lm, ('.',), '</s>')
-2.451695969857692
```

In [ ]:
def log_prob(lm, context, word):
    ## YOUR CODE HERE
    # Use log with base 2!
    return 0

print(log_prob(toy_lm, ('.',), '</s>'))

### (c7) Perplexity (16 points)

Our final step to make our langauge model functional would be to calculate perplexity of a document (e.g., a new text file). Implement the function `get_ppl(lm, newfile)` that returns the perplexity of `newfile` given language model `lm` that you built using `build_ngram_model`.

Be mindful to check whether a word appears in the language model's vocabulary; if not, replace with `<UNK>`.

```
>>> get_ppl(toy_lm, "data/test_short.txt")
>>> 51.98841239479734
```

In [ ]:
def get_ppl(lm, testfile):
    ## YOUR CODE HERE
    return 0

get_ppl(toy_lm, "data/test_short.txt")

## (d) Authorship attribution (10 points)

If we have built language models of multiple authors, they can be used to check the author of an unknown piece of writing. Concretely, for the unknown text, we can run the file through each known author's language model, and use perplexity to predict which author the unknown text is most likely to belong to.

In this problem, build two **bigram** models:
- a  Shakespeare language model
- a  Jane Austen language model

Make sure to train these models from the full text. Once you have trained both models, use the `get_ppl` function from each language model on the file `test.txt`. 

Who wrote the text?

## (e) Bonus: Random text generator (5 points)

Now, we are ready to generate some Shakespeare text! Starting with the sentence start symbol `<s>`, at each step, use the previously generated word as context, and sample one of the top 5 words that follow this word. We stop whenever we hit `</s>`, or when we have generated a 20-word sentence, whichever is earlier.

Implement a function `text_generator(bigramlm, k)` that takes a bigram langauge model---trained on our Shakespeare text---as input, and generates `k` random sentences.

In [ ]:
def text_generator(bigramlm, k):
    